In [2]:
import os
import glob
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

# BGE-M3 모델 로드 (BAAI/bge-m3 사용)
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# 지정된 폴더와 내부 폴더 경로 설정
root_dir = './COMPANY_DATA'  # 지정된 폴더 경로 (직급별 폴더들)

# Chroma 벡터 DB 초기화
db_dir = './chroma_db'
vectorstore = Chroma(persist_directory=db_dir, embedding_function=embedding_model)

# 청킹을 위한 텍스트 분할기 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

# 폴더 순회 및 파일 처리
def process_files_in_folder(folder_path: str, role: str):
    # 지정된 폴더 내의 모든 텍스트 파일 찾기
    txt_files = glob.glob(os.path.join(folder_path, "*.txt"))

    for txt_file in txt_files:
        print(f"Processing file: {txt_file}")
        with open(txt_file, 'r', encoding='utf-8') as f:
            text = f.read()

        # 문서 청킹
        chunks = text_splitter.split_text(text)

        # 청크를 Document 객체로 변환
        documents = []
        for chunk in chunks:
            doc = Document(
                page_content=chunk,
                metadata={
                    "sourcefile": os.path.basename(txt_file),  # 파일명
                    "role": role,  # 직급 권한 (폴더 이름)
                    "last_edit": "2025-08-19"  # 마지막 수정일 고정
                }
            )
            documents.append(doc)

        # 배치로 문서들 추가 (올바른 메서드 사용)
        if documents:
            vectorstore.add_documents(documents)
            print(f"Added {len(documents)} chunks from {os.path.basename(txt_file)}")

# 루트 폴더 내의 모든 직급 폴더 처리
def process_root_folder(root_dir: str):
    for role_folder in os.listdir(root_dir):
        role_folder_path = os.path.join(root_dir, role_folder)
        if os.path.isdir(role_folder_path):  # 내부 폴더만 처리
            print(f"Processing role folder: {role_folder}")
            process_files_in_folder(role_folder_path, role_folder)

# 데이터 처리 시작
if __name__ == "__main__":
    try:
        process_root_folder(root_dir)
        print("모든 문서가 성공적으로 벡터 데이터베이스에 저장되었습니다.")
    except Exception as e:
        print(f"오류 발생: {e}")

Processing role folder: assistant
Processing file: ./COMPANY_DATA\assistant\01_프로젝트_관리_문서__M-Core_프로젝트_진행_현황_보고서.txt
Added 4 chunks from 01_프로젝트_관리_문서__M-Core_프로젝트_진행_현황_보고서.txt
Processing file: ./COMPANY_DATA\assistant\02_프로젝트_관리_문서__M-Core_프로젝트_일정_관리_문서.txt
Added 3 chunks from 02_프로젝트_관리_문서__M-Core_프로젝트_일정_관리_문서.txt
Processing file: ./COMPANY_DATA\assistant\03_프로젝트_관리_문서__M-Core_프로젝트_리스크_관리_보고서.txt
Added 3 chunks from 03_프로젝트_관리_문서__M-Core_프로젝트_리스크_관리_보고서.txt
Processing file: ./COMPANY_DATA\assistant\04_프로젝트_관리_문서__M-Core_프로젝트_품질_점검_보고서.txt
Added 3 chunks from 04_프로젝트_관리_문서__M-Core_프로젝트_품질_점검_보고서.txt
Processing file: ./COMPANY_DATA\assistant\05_팀_성과_자료__콘텐츠팀_KPI_달성_현황.txt
Added 4 chunks from 05_팀_성과_자료__콘텐츠팀_KPI_달성_현황.txt
Processing file: ./COMPANY_DATA\assistant\06_팀_성과_자료__영업지원팀_팀_단위_성과_분석.txt
Added 4 chunks from 06_팀_성과_자료__영업지원팀_팀_단위_성과_분석.txt
Processing file: ./COMPANY_DATA\assistant\07_팀_성과_자료__고객지원팀_팀_성과_종합_보고서.txt
Added 4 chunks from 07_팀_성과_자료__고객지원팀_팀_성과_종합_보고서.txt
Processi

In [3]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# BGE-M3 모델 로드 (BAAI/bge-m3 사용)
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# Chroma 벡터 DB 로드
db_dir = './chroma_db'
vectorstore = Chroma(persist_directory=db_dir, embedding_function=embedding_model)

# 검색할 쿼리
query = "2025 결산 보고서의 법적 리스크 관리에서 법무팀과의 협조체계는 어떻게 구성되어 있나요?"

# 벡터 DB에서 유사한 문서 검색
results = vectorstore.similarity_search(query, k=3)  # k는 검색할 상위 결과의 수 (예: 3)

# 검색된 결과 출력
for i, result in enumerate(results):
    print(f"Result {i+1}:")
    print(f"Content: {result.page_content}")
    print(f"Metadata: {result.metadata}")
    print("=" * 50)


Result 1:
Content: 4) 예산 계획(각 부문별 예산 및 리소스 배분)
2025년 예산은 연구개발 부문에 집중 배정하여 기술 혁신과 신규 콘텐츠 개발에 힘쓸 것입니다. 글로벌 확장을 위한 마케팅 비용은 전년 대비 15% 증액하며, 인력 자원을 효율적으로 배분하여 각 부서의 성과 최대화를 목표로 합니다. 부장급 이상의 관리자는 각 부문별 예산이 계획대로 집행되는지 지속적으로 점검해야 합니다.

5) 리스크 관리 계획(위험 요소 및 대응 방안)
주요 리스크 요소로는 기술 개발 일정 지연, 시장 경쟁 심화, 법적 규제 변화가 있습니다. 이에 대한 대응 방안으로 기술 개발의 체계적인 관리 및 외부 파트너와의 협력 채널 강화가 필요합니다. 경쟁 심화에 대응하기 위해 차별화된 고객 경험을 제공할 것이며, 법적 규제 변화는 전담 법무팀을 통해 사전 대응 방안을 마련할 것입니다.
Metadata: {'role': 'senior_manager', 'last_edit': '2025-08-19', 'sourcefile': '02_전사_전략_문서__경영진_회의_자료_(2025년_Q1).txt'}
Result 2:
Content: 5) 리스크 관리 및 대응 방안  
올해 주요 리스크는 시장 불확실성과 기술 변화로 인한 경쟁 심화입니다. 이런 리스크에 대비하기 위해 시장 모니터링 체계를 강화하고, 주요 이해관계자와의 소통을 통해 발 빠른 대응을 준비하고 있습니다. 또한, 리스크 관리 프로세스를 정기적으로 점검하여 대응 전략의 유효성을 지속적으로 평가할 것입니다.

6) 변경 이력(v1.0 — 2025-08-19)  
첫 번째 버전(v1.0)에서는 부서별 예산 배분 계획과 주요 목표를 설정하였습니다. 향후 재무 실적에 따라 추가 조정이 필요할 수 있으며, 중요 변경 사항 발생 시 다시 보고드리겠습니다.
Metadata: {'sourcefile': '04_재무_관련_문서__2025_연간_예산안.txt', 'role': 'senior_manager', 'last_edit': '